
# Set up by:
1. instal updated packages
2. load packages
3. load key
4. load open ai key




In [ ]:
#install updated packages
#!pip install openai
!pip install --upgrade openai
!pip install --upgrade backoff
!pip install --upgrade python-dotenv

# load packages
import openai
import backoff
import os
import IPython
import pandas as pd

import time
from tqdm import tqdm


#load key

# openai.api_key  = ''




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
#check to see what are avaiable model to this api key
import pprint

def list_all_models():
    model_list = openai.Model.list()['data']
    model_ids = [x['id'] for x in model_list if 'gpt' in x['id']]
    model_ids.sort()
    pprint.pprint(model_ids)

if __name__ == '__main__':
    list_all_models()

['gpt-3.5-turbo',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-4',
 'gpt-4-0314',
 'gpt-4-0613']



# Func for cleaning data

In [ ]:
# @title
# Functions to process data
def num_to_label(num_label):
    '''
    convert label 0.1.2. to text-based label
    '''
    if num_label == 0:
        label = 'Passive'
    elif num_label == 1:
        label = 'Active'
    elif num_label == 2:
        label = 'Constructive'
    else:
        print('Undefined label found: ', num_label)
        exit()

    return label

def label_to_num(text_label):
    '''
    convert label 0.1.2. to text-based label
    '''
    if 'passive' in text_label.lower():
        label = 0
    elif 'active' in text_label.lower():
        label = 1
    elif 'constructive' in text_label.lower() :
        label = 2
    else:
        print('Undefined label found: ', text_label)
        label = 3

    return label


def extract_pred_label_COT(response):
  '''
  When we use the following format (used in pureFew shot)
   "Question:
    "Statement:
    "Label:
    "Chain-of-thought:
  '''
  splitted_res = response.splitlines()

  label_num = 3 #undefined
  for text in splitted_res:
    if "label:" in text.lower():
      label_num = label_to_num(text)
      break

  return label_num


def clean_text(text):
    '''
    Remove new liens / "" in the students response
    remove extra space
    '''
    # print("Before cleaning: ", text)
    text = text.strip()
    text = text.strip('\"')
    text = text.strip('\n')

    # put a period at the end of the sentence if not.
    text = add_period(text)

    text = text.capitalize()


    # print("After cleaning: ", text)

    return text

def add_period(text):
    if text == "":                    # Don't change empty strings.
        return text
    if text[-1] in ["?", ".", "!"]:   # Don't change if already okay.
        return text
    if text[-1] == ",":               # Change trailing ',' to '.'.
        return text[:-1] + "."
    return text + "."                 # Otherwise, add '.'.

# Confusion matrix (moved to eval_measure)

In [ ]:
# @title
from sklearn.metrics import confusion_matrix
import seaborn as sn

import matplotlib.pyplot as plt
import numpy as np

def show_confusionmatrix(y_true,y_pred):

    '''
    recall_list, prec_list, count_list = show_confusionmatrix(y_true,y_pred,classes)
    '''

    # classes = (0,1,2)
    classes = tuple(set(y_pred))


    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm_recall = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])

    df_cm_prec = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=0)[None,:], index = [i for i in classes],
                        columns = [i for i in classes])

    print("Recall")
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm_recall, annot=True)
    plt.xlabel('Pred')
    plt.ylabel('True')
    # plt.savefig('cf_matrix_ratio.png')

    print("Prec")
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm_prec, annot=True)
    plt.xlabel('Pred')
    plt.ylabel('True')

    print("Count")
    plt.figure(figsize = (12,7))
    sn.heatmap(cf_matrix, annot=True)
    plt.xlabel('Pred')
    plt.ylabel('True')
    # plt.show()
    # plt.savefig('cf_matrix_num.png')

    # Store the numbers
    flatten_df_cm_recall = df_cm_recall.to_numpy().flatten().tolist()
    flatten_df_cm_prec = df_cm_prec.to_numpy().flatten().tolist()
    flatten_cf_matrix = cf_matrix.flatten().tolist()

    return flatten_df_cm_recall, flatten_df_cm_prec,  flatten_cf_matrix


def accuracy(y_true,y_pred):
    return sum(1 for x,y in zip(y_true,y_pred) if x == y) / float(len(y_true))




# Function for data iteration (def iterate_data())


In [ ]:
# loop the data-----------------------------------------------------
# test_df = df.head(3)

from google.colab import files

def iterate_data(df, gpt_model, prompt_ver, index_start=0, use_backoff=True):
  ids = []
  true_labels = []
  responses_text = []
  pred_labels = []
  correctness = []

  num_req = 0
  for index, row in tqdm(df.iterrows(), total=len(df)):
    if index >= index_start :
      num_req += 1
      # true_label = num_to_label(row.label)

      if prompt_ver == "1a_v1":
        prompt = gen_prompt_1a_v1(row.question, row.feature)
      elif prompt_ver == "2a_v1":
        prompt = gen_prompt_2a_v1(row.question, row.feature)
      elif prompt_ver == "1a_v2":
        prompt = gen_prompt_1a_v2(row.question, row.feature)
      elif prompt_ver == "2a_v2":
        prompt = gen_prompt_2a_v2(row.question, row.feature)
      elif prompt_ver == "1a_v1_assertion":
        prompt = gen_prompt_1a_v1_assertion(row.question, row.feature)
      elif prompt_ver == "1a_v2_assertion":
        prompt = gen_prompt_1a_v2_assertion(row.question, row.feature)
      elif prompt_ver == "1a_v2_assertion_dos_and_donts":
        prompt = gen_prompt_1a_v2_assertion_dos_and_donts(row.question, row.feature)
      elif prompt_ver == "pure_fewshot_specificCOT":
        prompt = gen_prompt_pure_fewshot_specificCOT(row.question, row.feature, general_cot=False, assertion=False)
      elif prompt_ver == "pure_fewshot_specificCOT_genCOT":
        prompt = gen_prompt_pure_fewshot_specificCOT(row.question, row.feature, general_cot=True, assertion=False)
      elif prompt_ver == "pure_fewshot_specificCOT_assertion":
        prompt = gen_prompt_pure_fewshot_specificCOT(row.question, row.feature, general_cot=False, assertion=True)
      elif prompt_ver == "pure_fewshot_specificCOT_genCOT_assertion":
        prompt = gen_prompt_pure_fewshot_specificCOT(row.question, row.feature, general_cot=True, assertion=True)
      else:
        print('No prompt is defined for {0}'.format(prompt_ver))


      try:
        if use_backoff:
          messages = [{"role": "user", "content": prompt}]
          completion = get_completion_with_backoff(model=gpt_model,messages=messages,temperature=0)
          response = completion.choices[0].message["content"]
        else:
          response = get_completion(prompt, model=gpt_model)

        ids.append(row.id_internal)
        true_labels.append(row.label)
        responses_text.append(response)

        if "pure_fewshot" in prompt_ver:
          pred_labels.append(extract_pred_label_COT(response)))

        else:
          pred_labels.append(label_to_num(response))

        correctness.append(row.label==label_to_num(response))

      except Exception as error:
        print("An exception occurred:", error)

        print("GPT stops at index {0}, propossed {1} number of data ".format(index, len(true_labels)))

        break

      if use_backoff and num_req >= 3:
        time.sleep(60)
        num_req = 0

  return index, ids, true_labels, responses_text, pred_labels, correctness

def gen_pred_csv(df, index_start, ids, true_labels, responses_text, pred_labels, correctness, output_csv):
  index_end = len(true_labels) + index_start
  sub_df = df.iloc[index_start:index_end]

  sub_df['id_for_check'] = ids
  sub_df['pred_response'] = responses_text
  sub_df['pred_label'] = pred_labels
  sub_df['correctness'] = correctness

  sub_df.to_csv('/content/drive/My Drive/Data_ClassificationModel/' + output_csv, index=False)
  # files.download(output_csv)

# Get_completion

In [ ]:
def get_completion(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=100)
def get_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

# 1a Experiment 1


In [ ]:
def gen_prompt_1a_v1(question, statement):

    contextual_text = "'''\nStep 0: Refer to the example provided to initiate the chain of thought process:\n"+\
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n"+\
    "Statement: I can tell if the person liked something or not\n"+\
    "Label: Passive \n"+\
    \
    "Step 1: Identify the initial cognitive engagement level of the text according to the cogniive engagment definitions below.\n"+\
    \
    "Step 2: Second, read the user statement and identify what cognitive engagement level it aligns with from the following engagement level category list. \n"+\
    "Engagement level category list: \n"+\
    "1. Passive engagement: Focuses on recalling or reproducing information without further analysis. It might present facts, ask for basic recall, or provide statements that expect verbatim understanding of the subject. \n"+\
    "2. Active engagement: Demonstrates an application of previously learned knowledge to familiar yet slightly different contexts. It often involves problem-solving based on previous experiences or instruction, and demonstrates the usage of acquired knowledge in a variety of familiar scenarios. \n"+\
    "3. Constructive engagement: Characterized by applying knowledge in entirely new ways, critiquing processes or ideas, or creatively problem-solving. They often involve interpreting, combining, or generating new ideas based on multiple concepts.\n"+\
    "Step 3: When you identify which cognitive engagment category the statement aligns with, analyze why it fits into that category step by step.\n" +\
    \
    "This example is passive because it focuses on recalling or reproducing information without further analysis.\n"+\
    "This example is not active because it does not show the application of learned knowledge to slightly different contexts or problem-solving based on previous experiences or learned information.\n"+\
    \
    "This example is not constructive because it does not involve applying knowledge in new ways, critiquing processes, or creatively problem-solving based on the synthesis of multiple concepts.\n"+\
    \
    "This example is passive but can be Active if we change this to: \n"+\
    "To figure out if someone liked or didn't like something from a review, I'd basically check out the words they use. Like, if there are lots of good words and compliments, they probably liked it. But if there's loads of bad words or they're complaining about stuff, then they don't like it.\n"+\
    \
    "Or it can be Constructive if we change this to:\n"+\
    "So like, one way to figure it out is to not just look at the words they use but like, the whole story they're telling in the review. Like, good reviews usually tell a happy story, leading up to all the awesome stuff. But bad reviews, they kinda like to list all the bad things one by one. Oh, tone" +\
    "plays an important role so it is important to pay attention to it  because sometimes people use good words but they're being sarcastic and stuff, which means it's actually a bad review. So yeah, you gotta really get into it and understand the whole thing to know what they're really saying.\n'''"


    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    text = '```' + test_sample +  '```\n'


    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"

    prompt = task_intro + text +  context_intro + contextual_text

    return prompt


In [ ]:
# test_sample = "Question: Why do people write reviews? \
# Statement: People write reviews to express their feelings on a certain thing to \
# condemn a praise a business, franchise, movie, or book."

question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."



prompt = gen_prompt_1a_v1(question, statement)
print(prompt)
# response = get_completion(prompt, model='gpt-3.5-turbo')
# response = get_completion(prompt, model='gpt-3.5-turbo-0613')
# response = get_completion(prompt, model='gpt-4')

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 0: Refer to the example provided to initiate the chain of thought process:
Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?
Statement: I can tell if the person liked something or not
Label: Passive 
Step 1: Identify the initial cognitive engagement level of the text according to the cogniive engagment definitions below.
Step 2: Second, read the user statement and identify what cognitive engagement level it

# 1a Experiement 2
Changed up wording

In [ ]:
def gen_prompt_1a_v2(question, statement):

    contextual_text = "'''\nStep 1: Read the question and the corresponding response attentively to understand the context and the nature of the response provided.\n"+\
  "Step 2: Determine the initial cognitive engagement level of the response using the definitions of the cognitive engagement categories - passive, active, and constructive.\n"+\
  "1. Passive engagement: the student statement seems to be recalling a generally accepted fact or reproducing information without further analysis.\n"+\
  "2. Active engagement: the students statement seems to apply previously learned knowledge or shows an application of  previously learned knowledge to slightly \different contexts  and or demonstrates the use of acquired knowledge in a variety of scenarios.\n"+\
  "3. Constructive engagement: the students' statement critically engages with the information, interpreting it and contrasting it with their own understanding and or generating new ideas  based on multiple.concepts through inference and judgment.\n"+\
  "Step 3: Assess why it corresponds to the initial category you placed it in. Consider the extent to which it demonstrates recall of basic information (passive), application of learned knowledge to slightly different contexts (active), or a deeper level of analysis and synthesis of various concepts (constructive).\n"+\
  "Step 4: Critically evaluate whether the response could potentially belong to other engagement categories. Examine the nuances of the response to see if there are elements that might indicate a higher or lower level of cognitive engagement.\n"+\
  'Step 5: To upgrade the response to a higher engagement level, propose alterations that would make it align with the criteria for the "Active" category. This could involve adding details that show the application of earned knowledge to familiar yet slightly different contexts, or demonstrating problem-solving based on previous experiences.\n'+\
  'Step 6: Explore how the response can be restructured to meet the criteria of the "Constructive" engagement category. Consider adding elements that showcase deeper analysis, critical evaluation, or synthesis of multiple concepts to create a more nuanced and thoughtful response.\n'+\
  "Step 7: Finally, revisit the question and answer text to evaluate the original cognitive engagement level making sure the prediction of cognitive engagement is accurate.\n"+\
  "For example: after reading the question and answer in Step 1 the Initial Classification for step 2 is Passive engagement. The statement is quite basic, focusing on a straightforward method of determining a review’s tone, without delving into the specifics of how this can be achieved.\n"+\
  "For step 3: The response showcases a passive engagement because it states a simple ability to discern whether a person liked something or not, but it does not illustrate how this can be done or apply any deeper" +\
  "understanding or strategy to the process. It merely expresses a basic recall of the capability to differentiate between positive and negative reviews without further explanation or analysis.\n"+\
  "After critically evaluating the unlabeled text the response is fundamentally passive, it has the potential to be expanded into active or constructive categories with further detail and demonstration of analytical or critical thinking.\n"+\
  "To upgrade it to a higher level of engagement like Active or Constructive the response would need to be revised as follows:\n"+\
  "Active: To figure out if someone liked or didn't like something from a review, I'd basically check out the words they use. Like, if there are lots of good words and compliments, they probably liked it. But if there's loads of bad words or they're complaining about stuff, then they don't like it.\n"+\
  "Constructive: So like, one way to figure it out is to not just look at the words they use but like, the whole story they're telling in the review. Like, good reviews usually tell a happy story, leading up to all the awesome stuff. "+\
  "But bad reviews, they kinda like to list all the bad things one by one. Oh, tone plays an important role so it is important to pay attention to it because sometimes people use good words but they're being sarcastic and stuff, which means it's actually a bad review. So yeah, you gotta really get into it"+ \
  "and understand the whole thing to know what they're really saying.\n"+\
  'Finally, when revisiting in step 7 it is apparent that it maintains a "Passive" cognitive engagement level as it just mentions a basic understanding without further analysis or showcasing any strategic approach to determining a review\'s positivity or negativity. The revised responses, '+\
  'on the other hand, showcase a deeper engagement with the question, representing "Active" and "Constructive" levels of cognitive engagement respectively, '+\
  "by progressively incorporating a more detailed analysis and critical evaluation of the different ways to analyze reviews.\n'''"

    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    # text = '```' + clean_text(str(test_sample))+  '```\n'

    text = '```' + test_sample +  '```\n'


    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"

    prompt = task_intro + text +  context_intro + contextual_text

    return prompt

In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_1a_v2(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 1: Read the question and the corresponding response attentively to understand the context and the nature of the response provided.
Step 2: Determine the initial cognitive engagement level of the response using the definitions of the cognitive engagement categories - passive, active, and constructive.
1. Passive engagement: the student statement seems to be recalling a generally accepted fact or reproducing information without further analysis.
2. Active engageme

# 2a-Experiement 1

In [ ]:
def gen_prompt_2a_v1(question, statement):

    contextual_text = "'''\nStep 1: First read the user question to understand what was asked of the user. \n"+\
    "Step 2: Second, read the user statement and identify what cognitive engagement level it aligns with from the following engagement level category list. \n"+\
    "Engagement level category list: \n"+\
    "1. Passive engagement: Focuses on recalling or reproducing information without further analysis. It might present facts, ask for basic recall, or provide statements that expect verbatim understanding of the subject. \n"+\
    "2. Active engagement: Demonstrates an application of previously learned knowledge to familiar yet slightly different contexts. It often involves problem-solving based on previous experiences or instruction, and demonstrates the usage of acquired knowledge in a variety of familiar scenarios. \n"+\
    "3. Constructive engagement: Characterized by applying knowledge in entirely new ways, critiquing processes or ideas, or creatively problem-solving. They often involve interpreting, combining, or generating new ideas based on multiple + concepts.\n"+\
    "Step 3: When you identify which cognitive engagment category the statement aligns with, analyze why it fits into that category step by step.\n'''"

    text = "Question: Why do people write reviews? \
    Statement: People write reviews to express their feelings on a certain thing to \
    condemn a praise a business, franchise, movie, or book."

    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    # text = '```' + clean_text(str(test_sample))+  '```\n'

    text = '```' + test_sample +  '```\n'


    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"

    prompt = task_intro + text +  context_intro + contextual_text

    return prompt


In [ ]:
# test_sample = "Question: Why do people write reviews? \
# Statement: People write reviews to express their feelings on a certain thing to \
# condemn a praise a business, franchise, movie, or book."

question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_2a_v1(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 1: First read the user question to understand what was asked of the user. 
Step 2: Second, read the user statement and identify what cognitive engagement level it aligns with from the following engagement level category list. 
Engagement level category list: 
1. Passive engagement: Focuses on recalling or reproducing information without further analysis. It might present facts, ask for basic recall, or provide statements that expect verbatim understanding of the

# 2a Experiment 2

In [ ]:
def gen_prompt_2a_v2(question, statement):

    contextual_text = "'''\nStep 1: First read the user question to understand what is being asked of the user.\n" +\
    "Step 2: Second, read the user answer and identify what cognitive engagement level it best aligns with from the following engagement level category list. below. The answer can have more than one engagement level choose the highest level.\n" +\
    "Engagement level category list:\n" +\
    "1. Passive engagement: Focuses on recalling or reproducing information without further analysis. Passive engagment present facts, ask for basic recall, or provide statements that expect verbatim understanding of the subject.\n" +\
    "2. Active engagement: Demonstrates an application of previously learned knowledge to familiar yet slightly different contexts. It often involves problem-solving based on instruction, and demonstrates the usage of acquired knowledge in a variety of familiar scenarios.\n" +\
    "3. Constructive engagement: Characterized by applying knowledge in entirely new ways, critiquing processes or ideas, or creatively problem-solving. They often involve interpreting, combining, or generating new ideas based on multiple concepts.\n" +\
    "Step 3: When you identify which cognitive engagement category the statement aligns with, analyze why it fits into that category step by step.\n'''"


    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    # text = '```' + clean_text(str(test_sample))+  '```\n'

    text = '```' + test_sample +  '```\n'


    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"

    prompt = task_intro + text +  context_intro + contextual_text

    return prompt

In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_2a_v2(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 1: First read the user question to understand what is being asked of the user.
Step 2: Second, read the user answer and identify what cognitive engagement level it best aligns with from the following engagement level category list. below. The answer can have more than one engagement level choose the highest level.
Engagement level category list:
1. Passive engagement: Focuses on recalling or reproducing information without further analysis. Passive engagment pre

# 1a Experiment1 Assertion


In [ ]:
def gen_prompt_1a_v1_assertion(question, statement):

    contextual_text = "'''\nStep 0: Refer to the example provided to initiate the chain of thought process:\n"+\
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n"+\
    "Statement: I can tell if the person liked something or not\n"+\
    "Label: Passive \n"+\
    \
    "Step 1: Identify the initial cognitive engagement level of the text according to the cogniive engagment definitions below.\n"+\
    \
    "Step 2: Second, read the user statement and identify what cognitive engagement level it aligns with from the following engagement level category list. \n"+\
    "Engagement level category list: \n"+\
    "1. Passive engagement: Focuses on recalling or reproducing information without further analysis. It might present facts, ask for basic recall, or provide statements that expect verbatim understanding of the subject. \n"+\
    "2. Active engagement: Demonstrates an application of previously learned knowledge to familiar yet slightly different contexts. It often involves problem-solving based on previous experiences or instruction, and demonstrates the usage of acquired knowledge in a variety of familiar scenarios. \n"+\
    "3. Constructive engagement: Characterized by applying knowledge in entirely new ways, critiquing processes or ideas, or creatively problem-solving. They often involve interpreting, combining, or generating new ideas based on multiple concepts.\n"+\
    "Step 3: When you identify which cognitive engagment category the statement aligns with, analyze why it fits into that category step by step.\n" +\
    \
    "This example is passive because it focuses on recalling or reproducing information without further analysis.\n"+\
    "This example is not active because it does not show the application of learned knowledge to slightly different contexts or problem-solving based on previous experiences or learned information.\n"+\
    \
    "This example is not constructive because it does not involve applying knowledge in new ways, critiquing processes, or creatively problem-solving based on the synthesis of multiple concepts.\n"+\
    \
    "This example is passive but can be Active if we change this to: \n"+\
    "To figure out if someone liked or didn't like something from a review, I'd basically check out the words they use. Like, if there are lots of good words and compliments, they probably liked it. But if there's loads of bad words or they're complaining about stuff, then they don't like it.\n"+\
    \
    "Or it can be Constructive if we change this to:\n"+\
    "So like, one way to figure it out is to not just look at the words they use but like, the whole story they're telling in the review. Like, good reviews usually tell a happy story, leading up to all the awesome stuff. But bad reviews, they kinda like to list all the bad things one by one. Oh, tone" +\
    "plays an important role so it is important to pay attention to it  because sometimes people use good words but they're being sarcastic and stuff, which means it's actually a bad review. So yeah, you gotta really get into it and understand the whole thing to know what they're really saying.\n'''"


    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    text = '```' + test_sample +  '```\n'

    asssertaion_text = '- If a statement contains between one to four words, indicating a limited engagement with the topic, it should be labeled as "Passive." \n' + \
    \
    '- If a statement is presenting an opinion without providing detailed reasoning or analysis, it should be labeled as "Passive."\n' + \
    \
    '- If a statement presents general claims or facts without showcasing a deeper understanding or analysis should be considered "Passive."\n' + \
    \
    '- If a statement provides a surface-level hypothesis without delving deep into analysis or explaining the underlying reasoning, it should be labeled as "Passive."\n' + \
    \
    '- If a statement is mainly focused on recalling information or making simple comparisons without further analysis, it should be labeled as "Passive."\n' + \
    \
    '- If a statement applies a generally known concept to analyze elements, challenges, or features within a familiar context without delving into a deeper analysis or synthesis, it should be labeled as "Active."\n' + \
    \
    '- If a statement identifies a challenge or problem by applying basic knowledge without offering deeper insights or solutions, it should be labeled as "Active."\n' + \
    \
    '- If a statement analyzes and interprets the potential reasons behind certain observations, leveraging a deeper understanding of the subject matter, it should be labeled as "Constructive."\n' + \
    \
    '- If a statement hypothesizes about a phenomenon based on an understanding of underlying processes and the application of knowledge in new ways, it should be labeled as "Constructive."\n'

    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"
    assertion_intro = "\nA few facts about identifying the cogntive engagment level that you must assert while determining the level of engagement for the unlabeled text provided:\n"

    prompt = task_intro + text +  context_intro + contextual_text + assertion_intro + asssertaion_text

    return prompt


In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_1a_v1_assertion(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 0: Refer to the example provided to initiate the chain of thought process:
Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?
Statement: I can tell if the person liked something or not
Label: Passive 
Step 1: Identify the initial cognitive engagement level of the text according to the cogniive engagment definitions below.
Step 2: Second, read the user statement and identify what cognitive engagement level it

# 1a Experiment2 Assertion

In [ ]:
def gen_prompt_1a_v2_assertion(question, statement):

    contextual_text = "'''\nStep 1: Read the question and the corresponding response attentively to understand the context and the nature of the response provided.\n"+\
  "Step 2: Determine the initial cognitive engagement level of the response using the definitions of the cognitive engagement categories - passive, active, and constructive.\n"+\
  "1. Passive engagement: the student statement seems to be recalling a generally accepted fact or reproducing information without further analysis.\n"+\
  "2. Active engagement: the students statement seems to apply previously learned knowledge or shows an application of  previously learned knowledge to slightly \different contexts  and or demonstrates the use of acquired knowledge in a variety of scenarios.\n"+\
  "3. Constructive engagement: the students' statement critically engages with the information, interpreting it and contrasting it with their own understanding and or generating new ideas  based on multiple.concepts through inference and judgment.\n"+\
  "Step 3: Assess why it corresponds to the initial category you placed it in. Consider the extent to which it demonstrates recall of basic information (passive), application of learned knowledge to slightly different contexts (active), or a deeper level of analysis and synthesis of various concepts (constructive).\n"+\
  "Step 4: Critically evaluate whether the response could potentially belong to other engagement categories. Examine the nuances of the response to see if there are elements that might indicate a higher or lower level of cognitive engagement.\n"+\
  'Step 5: To upgrade the response to a higher engagement level, propose alterations that would make it align with the criteria for the "Active" category. This could involve adding details that show the application of earned knowledge to familiar yet slightly different contexts, or demonstrating problem-solving based on previous experiences.\n'+\
  'Step 6: Explore how the response can be restructured to meet the criteria of the "Constructive" engagement category. Consider adding elements that showcase deeper analysis, critical evaluation, or synthesis of multiple concepts to create a more nuanced and thoughtful response.\n'+\
  "Step 7: Finally, revisit the question and answer text to evaluate the original cognitive engagement level making sure the prediction of cognitive engagement is accurate.\n"+\
  "For example: after reading the question and answer in Step 1 the Initial Classification for step 2 is Passive engagement. The statement is quite basic, focusing on a straightforward method of determining a review’s tone, without delving into the specifics of how this can be achieved.\n"+\
  "For step 3: The response showcases a passive engagement because it states a simple ability to discern whether a person liked something or not, but it does not illustrate how this can be done or apply any deeper" +\
  "understanding or strategy to the process. It merely expresses a basic recall of the capability to differentiate between positive and negative reviews without further explanation or analysis.\n"+\
  "After critically evaluating the unlabeled text the response is fundamentally passive, it has the potential to be expanded into active or constructive categories with further detail and demonstration of analytical or critical thinking.\n"+\
  "To upgrade it to a higher level of engagement like Active or Constructive the response would need to be revised as follows:\n"+\
  "Active: To figure out if someone liked or didn't like something from a review, I'd basically check out the words they use. Like, if there are lots of good words and compliments, they probably liked it. But if there's loads of bad words or they're complaining about stuff, then they don't like it.\n"+\
  "Constructive: So like, one way to figure it out is to not just look at the words they use but like, the whole story they're telling in the review. Like, good reviews usually tell a happy story, leading up to all the awesome stuff. "+\
  "But bad reviews, they kinda like to list all the bad things one by one. Oh, tone plays an important role so it is important to pay attention to it because sometimes people use good words but they're being sarcastic and stuff, which means it's actually a bad review. So yeah, you gotta really get into it"+ \
  "and understand the whole thing to know what they're really saying.\n"+\
  'Finally, when revisiting in step 7 it is apparent that it maintains a "Passive" cognitive engagement level as it just mentions a basic understanding without further analysis or showcasing any strategic approach to determining a review\'s positivity or negativity. The revised responses, '+\
  'on the other hand, showcase a deeper engagement with the question, representing "Active" and "Constructive" levels of cognitive engagement respectively, '+\
  "by progressively incorporating a more detailed analysis and critical evaluation of the different ways to analyze reviews.\n'''"

    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    # text = '```' + clean_text(str(test_sample))+  '```\n'

    text = '```' + test_sample +  '```\n'

    asssertaion_text = '- If a statement contains between one to four words, indicating a limited engagement with the topic, it should be labeled as "Passive."\n' + \
    \
    '- If a statement is presenting an opinion without providing detailed reasoning or analysis, it should be labeled as "Passive."\n' + \
   \
    '- If a statement presents general claims or facts without showcasing a deeper understanding or analysis should be considered "Passive."\n' + \
   \
    '- If a statement provides a surface-level hypothesis without delving deep into analysis or explaining the underlying reasoning, it should be labeled as "Passive."\n' + \
   \
    '- If a statement is mainly focused on recalling information or making simple comparisons without further analysis, it should be labeled as "Passive."\n' + \
   \
    '- If a statement applies a generally known concept to analyze elements, challenges, or features within a familiar context without delving into a deeper analysis or synthesis, it should be labeled as "Active."\n' + \
   \
    '- If a statement identifies a challenge or problem by applying basic knowledge without offering deeper insights or solutions, it should be labeled as "Active."\n' + \
   \
    '- If a statement analyzes and interprets the potential reasons behind certain observations, leveraging a deeper understanding of the subject matter, it should be labeled as "Constructive."\n' + \
   \
    '- If a statement hypothesizes about a phenomenon based on an understanding of underlying processes and the application of knowledge in new ways, it should be labeled as "Constructive."\n'

    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"
    assertion_intro = "\nA few facts about identifying the cogntive engagment level that you must assert while determining the level of engagement for the unlabeled text provided:\n"


    prompt = task_intro + text +  context_intro + contextual_text + assertion_intro + asssertaion_text

    return prompt


In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_1a_v2_assertion(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Step 1: Read the question and the corresponding response attentively to understand the context and the nature of the response provided.
Step 2: Determine the initial cognitive engagement level of the response using the definitions of the cognitive engagement categories - passive, active, and constructive.
1. Passive engagement: the student statement seems to be recalling a generally accepted fact or reproducing information without further analysis.
2. Active engageme

# 1a v2 Assertaion dos and don'ts

In [ ]:
def gen_prompt_1a_v2_assertion_dos_and_donts(question, statement):

    contextual_text = "'''\nPassive:\n" + \
    "Question: What features do you think are indicators of positive reviews?\n" + \
    "Statement: Words like love, excellent, greatest, amazing, enjoy, awesome, best.\n\n" + \
    \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: I can tell if the person liked something or not.\n\n" + \
    \
    "Question: When you click on the row, the feature in this review will be highlighted in the feature graph (like the one you have seen in the Light On Light Off activity).Which feature do you think is it?\n" + \
    "Statement: Because it's associated with positivity.\n\n" + \
    \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statement: I can use major words that people say in reviews first. Words like 'love,' 'hate,' 'bad,' 'delicious,' and more.\n\n" + \
    \
    "Active:\n" + \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statment: You can look at the data set and find words that really stand out to you or words that have a strong emotional connotation. You can also check the graphand the probability in terms of the features being used or how strongly they correlate with the result.\n\n" + \
    \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: One strategy that you can use to determine if a review is positive or negative is looking at diction, which is word choice, and how the words are being used.\n\n" + \
    \
    "Question: When you click on the row, the feature in this review will be highlighted in the feature graph (like the one you have seen in the Light On Light Off activity). Which feature do you think is it?\n" + \
    "Statement: Love is the most defining word in this review, if it were changed to 'hate' it would have a completely different meaning\n\n" + \
    \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statement: You can test multiple reviews with words that you think may be the features to determine if they are actually features.\n\n" + \
    \
    "Constructive:\n" + \
    "Question: Why do people write reviews?\n" + \
    "Statement: To share their experience of a certain product or service so that they can either warn or recommend it to people. Sharing experiences is important so that way others who have not experienced it can know what they are getting in to.\n\n" + \
    \
    "Question: if none of the 10 features are present in your review, try again with another review. If some of the 10 features are in your review, examine both your review and the feature graph. What do you think these features are?\n" + \
    "Statement: I think these features are key words and numbers. Like the example used the word 'love' which implies a positive reply. The numbers also because if you say 1 out of 10 that's bad but if you say 10 out of 10 that's good.\n\n" + \
    \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: If I am having a conversation with somebody it will be easy to detect if the review is good or bad by word choice and their tone. If they wrote it, I will be able to see key words that point in either a positive or negative direction.\n\n" + \
    \
    "Question: What kinds of reviews can make our world a better place?\n" + \
    "Statement: Some reviews that can make the world a better place is if it's a review about a foreign country then it can give some insight into what is happening within that country. Or even here in the United States, it can share what's happening within their state and let the rest of the world know.\n\n" + \
    \
    "Step 1: Read the question and the corresponding response attentively to understand the context and the nature of the response provided.\n"+\
    \
    "Step 2: Determine the initial cognitive engagement level of the response using the definitions of the cognitive engagement categories - passive, active, and constructive.\n"+\
    \
    '1. Passive engagement: a statement  is classified as "Passive" when the individual is only receiving information without interacting with it or adding anything to it. Passive engagement typically involves listening, reading, or receiving information without actively processing, manipulating, or reflecting upon it.\n'+\
    '2. Active engagement: a statement is classified as "Active" when it involves manipulation or input of information but does not generate new ideas or concepts based on prior knowledge. Active engagement typically involves doing something with the information, such as organizing, summarizing, or categorizing, without necessarily adding anything new or reflecting deeply on the content.\n'+\
    '3. Constructive engagement: a statement is classified as "Constructive" if it reflects reasoning, justification, or thoughtful consideration based on prior knowledge.\n'+\
    \
    "Step 3: Assess why it corresponds to the initial category you placed it in. Consider the extent to which it demonstrates recall of basic information (passive), application of learned knowledge to slightly different contexts (active), or a deeper level of analysis and synthesis of various concepts (constructive).\n"+\
    "Step 4: Critically evaluate whether the response could potentially belong to other engagement categories. Examine the nuances of the response to see if there are elements that might indicate a higher or lower level of cognitive engagement.\n"+\
    'Step 5: To upgrade the response to a higher engagement level, propose alterations that would make it align with the criteria for the "Active" category. This could involve adding details that show the application of earned knowledge to familiar yet slightly different contexts, or demonstrating problem-solving based on previous experiences.\n'+\
    'Step 6: Explore how the response can be restructured to meet the criteria of the "Constructive" engagement category. Consider adding elements that showcase deeper analysis, critical evaluation, or synthesis of multiple concepts to create a more nuanced and thoughtful response.\n'+\
    \
    "Step 7: Finally, revisit the question and answer text to evaluate the original cognitive engagement level making sure the prediction of cognitive engagement is accurate.\n"+\
    "For example: after reading the question and answer in Step 1 the Initial Classification for step 2 is Passive engagement. The statement is quite basic, focusing on a straightforward method of determining a review’s tone, without delving into the specifics of how this can be achieved.\n"+\
    "For step 3: The response showcases a passive engagement because it states a simple ability to discern whether a person liked something or not, but it does not illustrate how this can be done or apply any deeper" +\
    "understanding or strategy to the process. It merely expresses a basic recall of the capability to differentiate between positive and negative reviews without further explanation or analysis.\n"+\
    "After critically evaluating the unlabeled text the response is fundamentally passive, it has the potential to be expanded into active or constructive categories with further detail and demonstration of analytical or critical thinking.\n'''"

    # "To upgrade it to a higher level of engagement like Active or Constructive the response would need to be revised as follows:\n"+\
    # "Active: To figure out if someone liked or didn't like something from a review, I'd basically check out the words they use. Like, if there are lots of good words and compliments, they probably liked it. But if there's loads of bad words or they're complaining about stuff, then they don't like it.\n"+\
    # "Constructive: So like, one way to figure it out is to not just look at the words they use but like, the whole story they're telling in the review. Like, good reviews usually tell a happy story, leading up to all the awesome stuff. "+\
    # "But bad reviews, they kinda like to list all the bad things one by one. Oh, tone plays an important role so it is important to pay attention to it because sometimes people use good words but they're being sarcastic and stuff, which means it's actually a bad review. So yeah, you gotta really get into it"+ \
    # "and understand the whole thing to know what they're really saying.\n"+\
    # 'Finally, when revisiting in step 7 it is apparent that it maintains a "Passive" cognitive engagement level as it just mentions a basic understanding without further analysis or showcasing any strategic approach to determining a review\'s positivity or negativity. The revised responses, '+\
    # 'on the other hand, showcase a deeper engagement with the question, representing "Active" and "Constructive" levels of cognitive engagement respectively, '+\
    # "by progressively incorporating a more detailed analysis and critical evaluation of the different ways to analyze reviews.\n'''"

    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement)) + "\n"

    # text = '```' + clean_text(str(test_sample))+  '```\n'

    text = '```' + test_sample +  '```\n'

    asssertaion_text = "- Don't label the statement as Active when the statement shows active engagement with the information.\n" + \
    "- Don't label the statement as Active when the statement forms a hypothesis about why the model learned a  weight for a certain feature.\n" + \
    "- Don't label the statement as Active if they are forming an opinion about its usefulness, and providing reasoning for their opinion.\n" + \
    "- Do label the statement as Constructive when the statement provides reasoning for why the model learned a negative weight, relating it to the sentiment of the review.\n" + \
    "- Do label the statement as Constructive if they are forming an opinion about its usefulness, and providing reasoning for their opinion.\n" + \
    "- Do label the statement as Constructive when the statement provides their interpretation and reasoning to the question.\n" + \
    "- Do label the statement as Constructive when they form a hypothesis about why the model learned a  weight for a certain feature.\n" + \
    "- Do label the statement as Constructive when the statement shows active engagement with the information.\n"


    task_intro = "Your task is to identify the label of the sentence delimited by triple backticks. " +\
    "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"

    context_intro = "Use the following examples delimited by triple quotes to understand which label the sentence belongs to.\n"
    assertion_intro = "\nA few facts about identifying the cogntive engagment level that you must assert while determining the level of engagement for the unlabeled text provided:\n"


    prompt = task_intro + text +  context_intro + contextual_text + assertion_intro + asssertaion_text

    return prompt


In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_1a_v2_assertion_dos_and_donts(question, statement))

Your task is to identify the label of the sentence delimited by triple backticks. Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
```
Use the following examples delimited by triple quotes to understand which label the sentence belongs to.
'''
Passive:
Question: What features do you think are indicators of positive reviews?
Statement: Words like love, excellent, greatest, amazing, enjoy, awesome, best.

Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?
Statement: I can tell if the person liked something or not.

Question: When you click on the row, the feature in this review will be highlighted in the feature graph (like the one you have seen in the L

# Pure Few shot with example specific COT + General COT (if True) + Assertaion (if True)

In [ ]:
def gen_prompt_pure_fewshot_specificCOT(question, statement, general_cot=False, assertion=False):

    contextual_text = "'''\n" + \
    "Question: What features do you think are indicators of positive reviews?\n" + \
    "Statement: Words like love, excellent, greatest, amazing, enjoy, awesome, best.\n" + \
    "Label: Passive\n" + \
    "Chain-of-thought: because it is a direct response that involves recalling or listing words without further analysis or interaction.\n\n" + \
    \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: I can tell if the person liked something or not.\n" + \
    "Label: Passive\n" + \
    "Chain-of-thought: because it does not specify any strategies or reflection to distinguish between positive and negative sentiments.\n\n" + \
    \
    "Question: When you click on the row, the feature in this review will be highlighted in the feature graph (like the one you have seen in the Light On Light Off activity).Which feature do you think is it?\n" + \
    "Statement: Because it's associated with positivity.\n" + \
    "Label: Passive\n" + \
    "Chain-of-thought: because it is simple information without reflection without delving into specific details, analysis, or reflection.\n\n" + \
    \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statement: I can use major words that people say in reviews first. Words like 'love,' 'hate,' 'bad,' 'delicious,' and more.\n" + \
    "Label: Passive\n" + \
    "Chain-of-thought: because it only has recall words and delve into any analysis, reflection, or application.\n\n" + \
    \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statment: You can look at the data set and find words that really stand out to you or words that have a strong emotional connotation. You can also check the graphand the probability in terms of the features being used or how strongly they correlate with the result.\n" + \
    "Label: Active\n" + \
    "Chain-of-thought: because it summarizes and organizes the information in a broad manner\n\n" + \
    \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: One strategy that you can use to determine if a review is positive or negative is looking at diction, which is word choice, and how the words are being used.\n" + \
    "Label: Active\n" + \
    "Chain-of-thought: because it details a method of analyzing the word choice in reviews, demonstrating the application of acquired knowledge to assess sentiments.\n\n" + \
    \
    "Question: When you click on the row, the feature in this review will be highlighted in the feature graph (like the one you have seen in the Light On Light Off activity). Which feature do you think is it?\n" + \
    "Statement: Love is the most defining word in this review, if it were changed to 'hate' it would have a completely different meaning\n" + \
    "Label: Active\n" + \
    "Chain-of-thought: because it demonstrates the application and analysis of knowledge in a familiar context but does not generate new ideas.\n\n" + \
    \
    "Question: What is one strategy you can use to determine what features someone has used to build a classification model?\n" + \
    "Statement: You can test multiple reviews with words that you think may be the features to determine if they are actually features.\n" + \
    "Label: Active\n" + \
    "Chain-of-thought: because it demonstrates the application and manipulation of knowledge in a familiar context without generating new insights.\n\n" + \
    \
    "Question: Why do people write reviews?\n" + \
    "Statement: To share their experience of a certain product or service so that they can either warn or recommend it to people. Sharing experiences is important so that way others who have not experienced it can know what they are getting in to.\n" + \
    "Label: Constructive\n" + \
    "Chain-of-thought: because it provides an understanding and reasoning of the broader context and implications why sharing experience is importants.\n\n" + \
    \
    "Question: if none of the 10 features are present in your review, try again with another review. If some of the 10 features are in your review, examine both your review and the feature graph. What do you think these features are?\n" + \
    "Statement: I think these features are key words and numbers. Like the example used the word 'love' which implies a positive reply. The numbers also because if you say 1 out of 10 that's bad but if you say 10 out of 10 that's good.\n" + \
    "Label: Constructive\n" + \
    "Chain-of-thought: because it provides interpretation and application to generate insights about the potential features in reviews.\n\n" + \
   \
    "Question: What is one strategy you (as a human) can use to determine if a review is positive or negative?\n" + \
    "Statement: If I am having a conversation with somebody it will be easy to detect if the review is good or bad by word choice and their tone. If they wrote it, I will be able to see key words that point in either a positive or negative direction.\n" + \
    "Label: Constructive\n" + \
    "Chain-of-thought: because it demonstrates a depth of reasoning and reflection of how to detemine if a review is positive or negative.\n\n" + \
    \
    "Question: What kinds of reviews can make our world a better place?\n" + \
    "Statement: Some reviews that can make the world a better place is if it's a review about a foreign country then it can give some insight into what is happening within that country. Or even here in the United States, it can share what's happening within their state and let the rest of the world know.\n" + \
    "Label: Constructive\n" + \
    "Chain-of-thought: because it provides reflection, thoughtful consideration and reasoning about the societal value and potential impact of reviews in fostering global understanding and awareness.\n'''"


    chain_of_thought = "\nStep 1: Read the question and statement attentively to understand the context and the nature of the statement provided.\n" + \
        "Step 2: Determine the initial cognitive engagement level of the statement using the definitions of the provided cognitive engagement labels - passive, active, and constructive.\n"+ \
        '1. Passive engagement: a statement  is classified as "Passive" when the individual is only receiving information without interacting with it or adding anything to it. Passive engagement typically involves listening, reading, or receiving information without actively processing, manipulating, or reflecting upon it.\n' +\
        '2. Active engagement: a statement is classified as "Active" when it involves manipulation or input of information but does not generate new ideas \or concepts based on prior knowledge. Active engagement typically involves doing something with the information, such as organizing, summarizing, or categorizing, without necessarily adding anything new or reflecting deeply on the content.\n' +\
        '3. Constructive engagement: a statement is classified as "Constructive" if it reflects reasoning, justification, or thoughtful consideration based on prior knowledge.\n' +\
        \
        "Step 3: Assess why it corresponds to the label you placed it in. Consider the extent to which it demonstrates recall of basic information (passive), application of learned knowledge to slightly different contexts (active), or a deeper level of analysis and synthesis of various concepts (constructive).\n"+ \
        \
        "Step 4: Critically evaluate whether the statement could potentially belong to other labels. Examine the nuances of the statement to see if there are elements that might indicate a higher or lower level of cognitive engagement.\n" +\
        \
        'Step 5: To upgrade the statement to a higher engagement level, propose alterations that would make it align with the criteria for the "Active" category. This could involve adding details that show the application of learned knowledge to familiar yet slightly different contexts, or demonstrating problem-solving based on previous experiences.\n' +\
        \
        'Step 6: Explore how the statement can be restructured to meet the criteria of the "Constructive" engagement category. Consider adding elements that showcase deeper analysis, critical evaluation, or synthesis of multiple concepts to create a more nuanced and thoughtful response.\n' +\
        \
        'Step 7: Finally, revisit the question and statement to evaluate the original cognitive engagement level making sure the prediction of cognitive engagement is accurate.\n'


    assertion_text = \
      "- Don't label the statement as Active when the statement shows active engagement with the information.\n" + \
      "- Don't label the statement as Active when the statement forms a hypothesis about why the model learned a  weight for a certain feature.\n" + \
      "- Don't label the statement as Active if they are forming an opinion about its usefulness, and providing reasoning for their opinion.\n" + \
      "- Do label the statement as Constructive when the statement provides reasoning for why the model learned a negative weight, relating it to the sentiment of the review.\n" + \
      "- Do label the statement as Constructive if they are forming an opinion about its usefulness, and providing reasoning for their opinion.\n" + \
      "- Do label the statement as Constructive when the statement provides their interpretation and reasoning to the question.\n" + \
      "- Do label the statement as Constructive when they form a hypothesis about why the model learned a  weight for a certain feature.\n" + \
      "- Do label the statement as Constructive when the statement shows active engagement with the information.\n"

    test_sample = "\nQuestion: " + clean_text(str(question)) +  "\nStatement: " + clean_text(str(statement))  + "\nLabel: <Generate label>\nChain-of-thought: <Generate the chain-of-thought>" + "\n"

    text = '```' + test_sample +  '```\n'


    task_intro_1 = "Your task is to identify the label of the statement delimited by triple backticks\n"
    task_intro_2 = "Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. \n"
    context_intro = "Use the following examples delimited by triple quotes to understand which label the statement belongs to.\n"


    prompt = task_intro_1

    if general_cot:
      chain_of_thought_intro = "Read the instructions below:\n"
      prompt +=  chain_of_thought_intro + chain_of_thought

    prompt += task_intro_2 + text +  context_intro + contextual_text

    if assertion:
      assertion_intro = "\nA few facts about identifying the cogntive engagment level that you must assert while determining the level of engagement for the unlabeled text provided:\n"

      prompt += assertion_intro + assertion_text


    return prompt

In [ ]:
question = "Why do people write reviews? "
statement = "People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book."


print(gen_prompt_pure_fewshot_specificCOT(question, statement, general_cot=False, assertion=False))

Your task is to identify the label of the statement delimited by triple backticks
Based on your understanding of cognitive engagement and the labeled examples provided, determine the level of engagement for the unlabeled text provided. 
```
Question: Why do people write reviews?
Statement: People write reviews to express their feelings on a certain thing to condemn a praise a business, franchise, movie, or book.
Label: <Generate label>
Chain-of-thought: <Generate the chain-of-thought>
```
Use the following examples delimited by triple quotes to understand which label the statement belongs to.
'''
Question: What features do you think are indicators of positive reviews?
Statement: Words like love, excellent, greatest, amazing, enjoy, awesome, best.
Label: Passive
Chain-of-thought: because it is a direct response that involves recalling or listing words without further analysis or interaction.

Question: What is one strategy you (as a human) can use to determine if a review is positive or

# Iterate data

In [ ]:
from google.colab import drive

import pandas as pd
from google.colab import files
import io

'''
Data need to have the following columns:
"id_internal" : id for the example
"feature" : student's response in text
"question" : question in text
"label": True label of the student's response (0,1,2)
'''


'''
From Google Drive
'''
drive.mount('/content/drive')
# file_path = '/content/drive/MyDrive/Classification Model/COT/data/mergeddata_withQuestion_9_13_23.csv'

file_path = '/content/drive/MyDrive/Classification Model/COT/data/updated_test_wquestion_9_13_23.csv'
df = pd.read_csv(file_path)

'''
by Upload
'''
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded['EDM_master - EDM_master.csv']))

print(len(df))

Mounted at /content/drive
135


In [ ]:
df_sub = df.dropna().groupby('label').sample(n=5,random_state=10)
df_sub = df_sub.sort_values(by=['id_internal'])


df_sub

,id_internal,question,feature,ce_mean,RR,label
4,4,Which features are the most responsible for th...,average,0.333333,0.829731,0
7,7,What is one strategy you can use to determine ...,by using my mind and listening to specific wor...,0.166667,0.229907,0
10,10,Analyze the examples you pasted above. Why do ...,some words that have negativity is shame i thi...,0.809524,0.570095,1
12,12,Which features do you think are responsible fo...,the features that i think are responsible for ...,0.962963,1.202371,0
23,23,What is one strategy you can use to determine ...,by typing different reviews of varying complex...,0.409091,0.825729,1
24,24,What is one strategy you (as a human) can use ...,i usually look for certain words that have lot...,0.785714,0.824886,1
36,36,Why do you think the model learned a large pos...,i feel like it had to do with the words and ho...,0.920000,0.487643,2
61,61,What can you tell from the graph? Do you think...,the graph tells me that good can be used in bo...,1.111111,0.572131,2
65,65,What other features do you think would help di...,disgustrnwonderfulrnfantasticrnnicernrudern,0.730769,0.706324,0
69,69,if none of the 10 features are present in your...,i think these features are word like amazing a...,0.269231,0.997919,0


Iterate data FULL data

In [ ]:
'''
# List of all models implemented
# gpt_models = ['gpt-3.5-turbo', 'gpt-4']
# prompt_vers = ['1a_v1', '2a_v1','1a_v2', '2a_v2', '1a_v1_assertion','1a_v2_assertion', '1a_v2_assertion_dos_and_donts', \
#              'pure_fewshot_specificCOT', 'pure_fewshot_specificCOT_genCOT', 'pure_fewshot_specificCOT_assertion', 'pure_fewshot_specificCOT_genCOT_assertion']

'''
# gpt_models = ['gpt-4']
# prompt_vers = ['2a_v2']


gpt_models = ['gpt-4']
prompt_vers = [ 'pure_fewshot_specificCOT_genCOT']


use_backoff = True

index_stopped_df = pd.DataFrame(columns =['gptname', 'prompt_ver', 'index_start', 'index_end'])

index_start = 57

for gpt_model in gpt_models:
  for prompt_ver in prompt_vers:
    # use df.iloc[0:3] if you want to run with smaller data for testing.
    index_end, ids, true_labels, responses_text, pred_labels, correctness = iterate_data(df, gpt_model, prompt_ver, index_start, use_backoff)

    output_csv = prompt_ver + "_" + gpt_model + '_index' + str(index_start) + "_to_" + str(index_end)  + '.csv'
    gen_pred_csv(df, index_start, ids, true_labels, responses_text, pred_labels, correctness, output_csv)


    print([gpt_model, prompt_ver, index_start, index_end])
    index_stopped_df.loc[len(index_stopped_df.index)] = [gpt_model, prompt_ver, index_start, index_end]



100%|██████████| 135/135 [34:01<00:00, 15.12s/it]

['gpt-4', 'pure_fewshot_specificCOT_genCOT', 57, 134]



<ipython-input-4-b277cff19d1c>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-4-b277cff19d1c>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-4-b277cff19d1c>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [ ]:
# @title
gpt_models = ['gpt-4']
prompt_vers = ['pure_fewshot_specificCOT']
for gpt_model in gpt_models:
  for prompt_ver in prompt_vers:
    # use df.iloc[0:3] if you want to run with smaller data for testing.
    output_csv = prompt_ver + "_" + gpt_model + '_index' + str(index_start) + "_to_" + str(index_end)  + '.csv'
    gen_pred_csv(df_sub, index_start, ids, true_labels, responses_text, pred_labels, correctness, output_csv)
    print([gpt_model, prompt_ver, index_start, index_end])
    index_stopped_df.loc[len(index_stopped_df.index)] = [gpt_model, prompt_ver, index_start, index_end]

['gpt-4', 'pure_fewshot_specificCOT', 0, 110]


<ipython-input-21-78f9fdb85afe>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-21-78f9fdb85afe>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-21-78f9fdb85afe>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

Iterating Subset of data multiple time

In [ ]:
# For data_subset
gpt_models = ['gpt-4']
prompt_vers = [ 'pure_fewshot_specificCOT', 'pure_fewshot_specificCOT_genCOT', 'pure_fewshot_specificCOT_assertion', 'pure_fewshot_specificCOT_genCOT_assertion']

use_backoff = True

index_stopped_df = pd.DataFrame(columns =['gptname', 'prompt_ver', 'index_start', 'index_end'])

index_start = 0

# k is the number of runs
for k in range(0,1):
  for gpt_model in gpt_models:
    for prompt_ver in prompt_vers:

      index_end, ids, true_labels, responses_text, pred_labels, correctness = iterate_data(df_sub, gpt_model, prompt_ver, index_start, use_backoff)

      output_csv = prompt_ver + "_" + gpt_model + '_index' + str(index_start) + "_to_" + str(index_end)  +"_v" + str(k) + '.csv'


      gen_pred_csv(df_sub, index_start, ids, true_labels, responses_text, pred_labels, correctness, output_csv)


      print([gpt_model, prompt_ver, index_start, index_end])
      index_stopped_df.loc[len(index_stopped_df.index)] = [gpt_model, prompt_ver, index_start, index_end]



100%|██████████| 15/15 [05:47<00:00, 23.19s/it]
<ipython-input-16-b277cff19d1c>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-16-b277cff19d1c>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-16-b277cff19d1c>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

['gpt-4', 'pure_fewshot_specificCOT', 0, 110]


100%|██████████| 15/15 [06:04<00:00, 24.28s/it]
<ipython-input-16-b277cff19d1c>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-16-b277cff19d1c>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-16-b277cff19d1c>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

['gpt-4', 'pure_fewshot_specificCOT_genCOT', 0, 110]


100%|██████████| 15/15 [05:54<00:00, 23.63s/it]
<ipython-input-16-b277cff19d1c>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-16-b277cff19d1c>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-16-b277cff19d1c>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

['gpt-4', 'pure_fewshot_specificCOT_assertion', 0, 110]


100%|██████████| 15/15 [06:32<00:00, 26.16s/it]

['gpt-4', 'pure_fewshot_specificCOT_genCOT_assertion', 0, 110]



<ipython-input-16-b277cff19d1c>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['id_for_check'] = ids
<ipython-input-16-b277cff19d1c>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pred_response'] = responses_text
<ipython-input-16-b277cff19d1c>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [ ]:
print(index_stopped_df)


  gptname                       prompt_ver  index_start  index_end
0   gpt-4  pure_fewshot_specificCOT_genCOT            0         57


In [ ]:
print(index_end)
print(len(ids))
print(len(true_labels))
print(len(responses_text))
print(len(pred_labels))
print(len(correctness))
print(len(df.loc[index_start:index_end]))

110
15
15
15
15
15
111


In [ ]:
print('start')
time.sleep(3)
print('end')

start
end
